In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
cd ../../Pickled_from_mongo/

/home/jovyan/capstone-52/Pickled_from_mongo


In [4]:
ls

combined_eg_gulf_1k_sample.p  combined_eg_gulf_200k_sample.p


In [5]:
df = pd.read_pickle('../Pickled_from_mongo/combined_eg_gulf_200k_sample.p')

In [6]:
df.shape

(192936, 5)

In [7]:
df = df.drop(['cleaned_geo','cleaned_name'], axis=1)

In [8]:
#df = df.set_index(['cleaned_name'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192936 entries, 0 to 95683
Data columns (total 3 columns):
_id             192936 non-null object
cleaned_text    192936 non-null object
class           192936 non-null object
dtypes: object(3)
memory usage: 5.9+ MB


In [10]:
df.shape

(192936, 3)

In [11]:
df.sample(8)

,_id,cleaned_text,class
91124,5a2ce210204c9e0400cf24d6,هناك فين,EG
75081,5a2edb834be1040042c23313,شئ مؤسف أن تصل الأمور في الى هذا المنحدر ويصنفها أعضاء في الكونغرس الامريكي بأنها تبتلع المعونات والوضع السياسي والاقتص…,GULF
6218,5a2c7ae3204c9e0400cdd92c,بس أنا بجيب ورد وشوكولا في الوقت المناسب برده ),EG
94045,5a2dd94e204c9e0400cf303f,عايزة فستان فرحي اللي مش هفرحه ده يبقى لونه أسود مثلاً,EG
67129,5a2cbe6b204c9e0400cec71b,يا دينى,EG
93442,5a2dd94e204c9e0400cf2de4,نحن بقع من العتم لولا نور الله فينا,EG
31064,5a2e2e1a4be1040042c18722,ادرس,GULF
26649,5a2e1bca4be1040042c175e3,شهاب لانه الاسم فخم ورجولي اكثر,GULF


## Benchmark LSA with no stop words

### Label Encode the Categories


In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
df['class_numerical'] = le.fit_transform(df['class'])

In [14]:
df.class_numerical.unique()

array([0, 1])

### Prepare Document Term Matrix


### TFIDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1)

In [17]:
document_term_matrix_sps = tfidf_vectorizer.fit_transform(df.cleaned_text)

In [18]:
document_term_matrix_sps

<192936x223045 sparse matrix of type '<class 'numpy.float64'>'
	with 1807963 stored elements in Compressed Sparse Row format>

In [ ]:
dtm_array = np.asarray(document_term_matrix_sps) 

In [ ]:
dtm_array[0,:]

In [ ]:
#(tfidf * tfidf.T).A

_________________________________________________

In [ ]:
tfidf_vectorizer.get_feature_names()

In [ ]:
document_term_matrix_df = pd.DataFrame(document_term_matrix_sps.toarray(),
                                       index=df.index,
                                       columns=tfidf_vectorizer.get_feature_names())

In [ ]:
document_term_matrix_df.head()

In [ ]:
pd.concat([df.cleaned_text, document_term_matrix_df], axis=1).sample(10)

### Compute SVD of Document Term Matrix

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
n_components = 2
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [ ]:
svd_matrix = SVD.fit_transform(document_term_matrix_sps)


In [ ]:
SVD.explained_variance_ratio_


In [ ]:
svd_matrix[:4]


### Load SVD Matrix with Documents and Labels


In [ ]:
latent_semantic_analysis = pd.DataFrame(svd_matrix,
                                        index=df.index,
                                        columns=component_names)
latent_semantic_analysis['cleaned_text'] = df.cleaned_text
latent_semantic_analysis['class'] = df['class']

In [ ]:
latent_semantic_analysis.sample(10)


In [ ]:
vocabulary_loadings = pd.DataFrame(SVD.components_,
                                   index=component_names,
                                   columns=tfidf_vectorizer.get_feature_names()).T

In [ ]:
vocabulary_loadings['abs_component_1'] = np.abs(vocabulary_loadings.component_1)
vocabulary_loadings['abs_component_2'] = np.abs(vocabulary_loadings.component_2)

### LSA_Topic Modeling

### Display Top Terms for Each Component

Top 10 Terms for Component 1

In [ ]:
vocabulary_loadings.sort_values('abs_component_1',ascending=False).head(10)

Top 10 Terms for Component 2


In [ ]:
vocabulary_loadings.sort_values('abs_component_2',ascending=False).head(10)

### Plot Top Two Components


In [ ]:
plt.figure(figsize=(7,7))
pc_1 = latent_semantic_analysis['component_1'].values
pc_2 = latent_semantic_analysis['component_2'].values

plt.scatter(pc_1, pc_2, c=df['class_numerical'], cmap='rainbow')

plt.xlabel('First PC')
plt.ylabel('Second PC')
plt.axvline(linewidth=0.5)
plt.axhline(linewidth=0.5)
plt.xlim(-.1,1)
plt.ylim(-.5,1)

In [ ]:
plt.figure(figsize=(6,6))
pc_1 = latent_semantic_analysis['component_1'].values
pc_2 = latent_semantic_analysis['component_2'].values

strings = df['cleaned_text'].values
for i, (x, y) in enumerate(zip(pc_1, pc_2)): 
    plt.text(x,y,strings[i][:10])

plt.scatter(pc_1, pc_2, c=df['class_numerical'], cmap='rainbow')

plt.xlabel('First PC')
plt.ylabel('Second PC')
plt.axvline(linewidth=0.5)
plt.axhline(linewidth=0.5)
plt.xlim(-.1,1)
plt.ylim(-.1,1)

In [ ]:
plt.figure(figsize=(6,6))
pc_1 = latent_semantic_analysis['component_1'].values
pc_2 = latent_semantic_analysis['component_2'].values

plt.scatter(pc_1, pc_2, c=df['class_numerical'], cmap='rainbow')

plt.xlabel('First PC')
plt.ylabel('Second PC')
plt.axvline(linewidth=0.5)
plt.axhline(linewidth=0.5)
plt.xlim(-.01,.5)
plt.ylim(-.3,.6)

In [ ]:
eg_mask = latent_semantic_analysis['class'] == 'EG'


In [ ]:
latent_semantic_analysis[eg_mask]

In [ ]:
gulf_mask = latent_semantic_analysis['class'] == 'GULF'

In [ ]:
latent_semantic_analysis[gulf_mask]

In [ ]:
latent_semantic_analysis[(latent_semantic_analysis['class'] == 'EG') 
                         & (latent_semantic_analysis.component_2 > .050)]

In [ ]:
latent_semantic_analysis[(latent_semantic_analysis['class'] == 'GULF') 
                         & (latent_semantic_analysis.component_2 > .50)]

## Try `50` SVD components

In [ ]:
n_components = 50
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [ ]:
svd_matrix = SVD.fit_transform(document_term_matrix_sps)

In [ ]:
sum(SVD.explained_variance_ratio_)

In [ ]:
cumulative_explained_variance_eg_gulf = np.cumsum(SVD.explained_variance_ratio_)

In [ ]:
cumulative_explained_variance_eg_gulf

### Plot the cumulative sum of the explained variance ratio from the `50` SVD components 

In [ ]:
plt.figure(figsize=(20,10))

X = np.arange(1,51)
cumulative_explained_variance_eg_gulf = np.cumsum(SVD.explained_variance_ratio_)

plt.plot(X, cumulative_explained_variance_eg_gulf, '-o')
plt.bar(X, SVD.explained_variance_ratio_, align='center', alpha=0.5)

for i, j in zip(X, np.cumsum(SVD.explained_variance_ratio_)):
    plt.annotate(str(j.round(2)), xy=(i+.1,j-.01))
    
plt.xlabel('SVD components')
plt.ylabel('Explained variance')

In [ ]:
plt.plot(range(50), np.cumsum(SVD.explained_variance_ratio_), label='cumulative explained variance')
plt.legend()

#### Display Top Terms for Each Component 'topics'


In [ ]:
vocabulary_expression = pd.DataFrame(SVD.components_,
                                     index=component_names,
                                     columns=tfidf_vectorizer.get_feature_names()).T


In [ ]:

for i in range(1,11):
    vocabulary_expression['abs_component_{}'.format(i)] = np.abs(vocabulary_expression['component_{}'.format(i)])

In [ ]:
vocabulary_expression['abs_component_1'].sort_values(ascending=False).head(7)


In [ ]:
vocabulary_expression['abs_component_2'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_3'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_4'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_5'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_6'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_7'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_8'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_9'].sort_values(ascending=False).head(7)

In [ ]:
vocabulary_expression['abs_component_10'].sort_values(ascending=False).head(7)

### Search

#### Encode the search term using the same vectorizer


In [ ]:
search_sentences = [
{"sentence": "الثوره المصريه تحولت من ثورة شارع محدش يزعل", "class" : "EG"},
{"sentence": "نفسي اكون زيك بعرف اطنشك أو اخليك اخر حاجة و بعد كده اضحك عليك بكلمتين و انت تصدق كل مرة عادي", "class" : "EG"},
{"sentence": "بما أن أغلب اللي متابعني مش بقدر اوصلهم أغلب الوقت. . ف كل يوم هعمل تويته آخر اليوم اللي هيعمل لايك", "class" : "EG"},
{"sentence": "مقاومتنا للأشياء طلعت بتقل مع الزمن، مبقيناش نناهد ف حاجة.. و مش عشان أحنا جامدين قوي. هو حيلنا بس", "class" : "EG"},
{"sentence": "عارف ايه احلى حاجة حاصلة ليا انى منك وانت برضه بتجرى فيا انت اخر كل يوم باخدك ف حضنى وانت اول", "class" : "EG"},
{"sentence": "القاضى اللى حكم على المعتقلين بالاعدام هو هو نفس القاضى اللى هيراقب الانتخابات", "class" : "EG"},
{"sentence": "وفجأة تيجي سيرة حاجة في وسط الكلام تقلب عليك القديم والجديد وترسم في دماغك علامات استفهام مالهاش", "class" : "EG"},
{"sentence": "السنة اللي فاتت الاعلام الانجليزي قال المفروض بيب يعرف انه في البريمييرليج لازم يتأقلم و يلعب كورتنا", "class" : "EG"},
{"sentence": "حرب و قتال و ناس تموت و هذا الدلخ يقول سعيد و مثل أجواء كرة القدم ", "class" : "GULF"},
{"sentence": "من غباء الهلالي الدلخ اللي يفتخر بفوز فريقه من قيادة رئيس الحكام كلاتنبيرغ له سنه ماسنع الحكام السعوديين", "class" : "GULF"},
{"sentence": "شفتوا هوشة شيعان وغالي لو هي بين الهلاليين كان شفتوا هاشتاق كبر راسهم المنسم وكان جاك هذا الدلخ ", "class" : "GULF"},
{"sentence": "ذويه ارفضوا لانه عيار جمبازي مافيه شي وبليس مايكسر اماعينه يامال لضعفه قطو بو سبعة ارواح ", "class" : "GULF"},
{"sentence": "صج ياجماعه في سوال محيرني ليش المتان مافيهم النفسيه عكس الضعاف تقول خاشوقه ومنفس", "class" : "GULF"},
{"sentence": "لم نعاند التاريخ مسيو خاشوقه بل الواقع والعقلانية ابعدنا من التدمير والانفلات", "class" : "GULF"},
{"sentence": "أي والله وعندي عنه ابو خاشوقة أسرار لا تشرف قد أقولها اذا لم يلجم لسانه عن سب وطني", "class" : "GULF"},
{"sentence": "قبل ماتتكلمين يالطيبه افهمي السالفه ومنب ملزومه بسنابي اني اشرح كل شيء صارت بالتفصيل بس لانك قلق خل", "class" : "GULF"},
]

In [ ]:
search_sentences_df = pd.DataFrame(search_sentences)


In [ ]:
search_sentences_df.sample(4)

In [ ]:
search_sentences_df.sentence


In [ ]:
search_terms_encoded = tfidf_vectorizer.transform(search_sentences_df.sentence)

In [ ]:
search_terms_encoded_df = pd.DataFrame(search_terms_encoded.toarray(), 
                                       index=search_sentences_df.sentence, 
                                       columns=tfidf_vectorizer.get_feature_names())

In [ ]:
search_terms_encoded_df.head()

### Select a Random Search Term

In [ ]:
random_search_term_df = search_terms_encoded_df.sample(3)
random_search_term_df.head()

### Append the search term to the document term matrix


In [ ]:
random_search_term_df

In [ ]:
dtm_with_search_term.head()

In [ ]:
dtm_with_search_term.tail()

In [ ]:
random_search_term_df

In [ ]:
document_term_matrix_df.head()

In [ ]:
dtm_with_search_term = document_term_matrix_df.append(random_search_term_df)

In [ ]:
random_search_term_df

In [ ]:
dtm_with_search_term.head()

In [ ]:
dtm_with_search_term.tail()

In [ ]:
component_names

In [ ]:
svd_matrix[0]

In [ ]:
svd_df = pd.DataFrame(svd_matrix, 
                      index=dtm_with_search_term.index, 
                      columns=component_names)

In [ ]:
dtm_with_search_term[:2]

### Identify the Vector for our Search Term

In [ ]:
search_term_svd_vector = svd_df.loc[random_search_term_df.index]
search_term_svd_vector

### Use Cosine Similarity to Find the Most Similar Vector

In [ ]:
svd_df.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
svd_df['cosine_sim'] = cosine_similarity(svd_df, search_term_svd_vector)

In [ ]:
svd_df[['cosine_sim']].sort_values('cosine_sim', ascending=False).head(5)

## LDA

In [20]:
import lda

In [22]:
svd_array = np.asarray(svd_matrix, dtype='int64') 

NameError: name 'svd_matrix' is not defined

In [ ]:
type(svd_array)

In [21]:
model = lda.LDA(n_topics=3, n_iter=1000, random_state=1)

model.fit(svd_array)

topic_word = model.topic_word_
n_top_words = 3

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

NameError: name 'svd_array' is not defined